In [2]:
import numpy as np
import cv2
import sys
import time


In [22]:
# You can use this function by providing the marker corners, the pixel coordinates of the point of interest
#(x is increasing to the right and y is increasing downward)
# the length of the ArUco marker in m, the camera matrix, and the distortion coefficients. It will return the 
# distance from the ArUco marker to the point of interest in m.

def distance_to_aruco(marker_corners, point_of_interest, marker_length, camera_matrix, dist_coeffs):
    # Obtain ArUco pose estimation
    rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(marker_corners, marker_length, camera_matrix, dist_coeffs)
    
    # Convert pixel coordinates of the point of interest to camera coordinates
    homogeneous_pixel_coords = np.array([[point_of_interest[0]], [point_of_interest[1]], [1]])
    homogeneous_camera_coords = np.dot(np.linalg.inv(camera_matrix), homogeneous_pixel_coords)
    camera_coords = homogeneous_camera_coords / homogeneous_camera_coords[2]
    
    # Transform camera coordinates to marker coordinates
    rotation_matrix, _ = cv2.Rodrigues(rvec)
    marker_coords = np.dot(rotation_matrix.T, (camera_coords - tvec.T))
    
    # Compute the distance from marker origin to the transformed point
    distance = np.linalg.norm(marker_coords)
    
    return distance

In [23]:
# This function will take an image and overlay evenly spaced circles within a specified box area.

#Note: 
    # origin(0,0) of picture is at top-left corner
    # x-axis increases to the right
    # y-axis increases downward
    
def overlay_circles_on_image(image, num_points, circle_radius, circle_color, top_left, top_right, bottom_left, bottom_right):
    # Convert numpy array to PIL Image
    image_pil = Image.fromarray(image)
    draw = ImageDraw.Draw(image_pil)
    height = bottom_left[1] - top_left[1]
    width = top_right[0] - top_left[0]
    y_points = np.linspace(top_left[1], bottom_left[1], num_points).astype(int)
    x_points = np.linspace(top_left[0], top_right[0], num_points).astype(int)
    for y in y_points:
        for x in x_points:
            left_up_point = (x - circle_radius, y - circle_radius)
            right_down_point = (x + circle_radius, y + circle_radius)
            draw.ellipse([left_up_point, right_down_point], fill=circle_color)
    return np.array(image_pil)


In [24]:
# calculates the relative rotation angles between two markers represented by their rotation 
# vectors rvec1 and rvec2. ( vec1 and rvec2 come from pose_estimation function.

def angle_between_markers(rvec1, rvec2):
    # Calculate relative rotation vectors
    relative_rvec1 = rvec1 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)
    relative_rvec2 = rvec2 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)

    # Convert relative rotation vectors to rotation matrices
    rotation_matrix1, _ = cv2.Rodrigues(relative_rvec1)
    rotation_matrix2, _ = cv2.Rodrigues(relative_rvec2)

    # Compute the rotation difference between the two markers
    relative_rotation_matrix = np.dot(rotation_matrix2, np.linalg.inv(rotation_matrix1))
        
    # Convert rotation matrix to Euler angles
    relative_euler_angles, _ = cv2.Rodrigues(relative_rotation_matrix)
     
    # Convert to Angles in x, y, and z
    theta_x = np.arctan2(relative_rotation_matrix[2, 1], relative_rotation_matrix[2, 2])
    theta_y = np.arctan2(-relative_rotation_matrix[2, 0], np.sqrt(relative_rotation_matrix[2, 1]**2 + relative_rotation_matrix[2, 2]**2))
    theta_z = np.arctan2(relative_rotation_matrix[1, 0], relative_rotation_matrix[0, 0])
    
    # Convert to degrees
    theta_x_deg = np.degrees(theta_x)
    theta_y_deg = np.degrees(theta_y)
    theta_z_deg = np.degrees(theta_z)
    
    # Make it a single matrix
    Theta = np.array([theta_x_deg, theta_y_deg, theta_z_deg])
        
    return Theta

In [ ]:
# Finds the absolute distance between two aruco markers
def distance_between_markers(tvec1, tvec2):
    distance = np.linalg.norm(tvec1 - tvec2)
    return distance


In [ ]:
def draw_axis(img, rvec, tvec, camera_matrix, dist_coeffs, length):
    axis_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,length]]).reshape(-1,3)

    # Project axis points to the image plane
    img_points, _ = cv2.projectPoints(axis_points, rvec, tvec, camera_matrix, dist_coeffs)

    # Convert image points to integers
    img_points = np.round(img_points).astype(int)

    # Draw lines
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[1].ravel()), (0,0,255), 2)  # x-axis (red)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[2].ravel()), (0,255,0), 2)  # y-axis (green)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[3].ravel()), (255,0,0), 2)  # z-axis (blue)

    return img

In [ ]:
def aruco_display(corners, ids, rejected, image):  
	if len(corners) > 0:
		
		ids = ids.flatten()
		
		for (markerCorner, markerID) in zip(corners, ids):
			
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners
			
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
			
			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			print("[Inference] ArUco marker ID: {}".format(markerID))
			
	return image

In [ ]:
def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)  # Define aruco_dict here
    parameters = cv2.aruco.DetectorParameters()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Call aruco_display to draw markers and their IDs
    frame = aruco_display(corners, ids, rejected_img_points, frame)

    if len(corners) > 1:
        print("There are two or more markers")
        # Estimate pose for the first marker
        rvec1, tvec1, _ = cv2.aruco.estimatePoseSingleMarkers(corners[0], 0.175, matrix_coefficients, distortion_coefficients)

        # Estimate pose for the second marker
        rvec2, tvec2, _ = cv2.aruco.estimatePoseSingleMarkers(corners[1], 0.175, matrix_coefficients, distortion_coefficients)
        
        # Calculating Relative Position Vectors
        distance_vector = tvec1-tvec2;
        distance = distance_between_markers(tvec1.squeeze(), tvec2.squeeze())
        print("Distance between markers:", distance)
        print("Distance vector between markers:", distance_vector)
        
        # Calculating Relative Rotations
        Theta = angle_between_markers(rvec1.squeeze(), rvec2.squeeze())  
        #print(f"Relative Rotation Angle around X-axis (degrees): {Theta[0]}")
        #print(f"Relative Rotation Angle around Y-axis (degrees): {Theta[1]}")
        print(f"Relative Rotation Angle around Z-axis (degrees): {Theta[2]}")
        
        # Calculating distance to top-left point
        poin_of_interest = 
        distance_to_aruco = distance_to_aruco(corners, point_of_interest, 0.175, camera_matrix, dist_coeffs)
        
        
        cv2.aruco.drawDetectedMarkers(frame, corners)
        draw_axis(frame, rvec1, tvec1, matrix_coefficients, distortion_coefficients, 0.1)
        draw_axis(frame, rvec2, tvec2, matrix_coefficients, distortion_coefficients, 0.1)

    return frame



In [27]:
import numpy as np
import cv2
from PIL import Image, ImageDraw

ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,}

aruco_type = "DICT_ARUCO_ORIGINAL"

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))


cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# While loop for computer vision, currently calculates distance between markers only
while cap.isOpened():
     
    ret, img = cap.read()
    
    # Calculates Position of Arucos and outputs the image with the axises and ids overlaid
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)
    
    # Add the Circle Overlay
    processed_image = overlay_circles_on_image(
    output,
    num_points=10,
    circle_radius=3,
    circle_color=(0, 0, 255, 255),
    top_left=(500, 100),  
    top_right=(1000, 100),   
    bottom_left=(500, 500),  
    bottom_right=(1000, 500) 
    )
    cv2.imshow('Estimated Pose',  processed_image)
   
    # Get the corners and ids of detected markers
    corners, ids, _ = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inferen